In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import pyodbc
import saspy

from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
sas = saspy.SASsession(cfgname='winlocal', results='html')

In [ ]:
query = """
SELECT pl.[description] as [Rendering_Provider],
pg.mstr_list_item_desc as Specialty,
b.[description] as CatDesc,
c.[description] as ActionDesc,
d.template_name,
a.event_timestamp,
a.event_message,
e.enc_nbr,
f.dxCode
FROM event_audit_mstr_combined as a INNER JOIN
tblkp_event_audit_mstr_category as b on a.category_id = b.category_id INNER JOIN
tblkp_event_audit_mstr_action as c ON a.action_id = c.action_id INNER JOIN
NGReport.dbo.templates as d ON a.template_id = d.template_id INNER JOIN
patient_encounter_compact as e ON a.encounter_id = e.enc_id INNER JOIN 
tbl_provider_list as pl on e.rendering_provider_id = pl.provider_id INNER JOIN
tbl_provider_group as pg ON pg.provider_id = pl.provider_id INNER JOIN
tbl_person as p ON a.person_id = p.person_id INNER JOIN
charges_compact as f ON e.enc_id = f.source_id
WHERE a.event_timestamp between '20190325' and '20190329'
AND e.billable_ind = 'Y'
AND pl.symed_active_flg = 'Y' 
AND pl.attending_ind = 'Y'
AND pg.group_id = 1
AND pg.mstr_list_item_desc = 'Endocrinology'

ORDER BY a.event_timestamp
"""

#AND f.dxcode = 'I10'

In [ ]:
cnxn = pyodbc.connect(driver='{SQL Server}', host='SRV-RH-REPORTS', database='CRHC_Report', trusted_connection='yes')
df = pd.read_sql(query, cnxn)
cnxn.close()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df['enc_nbr'] = df['enc_nbr'].astype(str)

In [ ]:
print(df['event_timestamp'].dt.date.nunique())
print(df['Rendering_Provider'].nunique())
df['dxCode'].nunique()
#df.assign(numrecs=1).groupby(['Rendering_Provider', 'enc_nbr'])['numrecs'].sum()
#df[['Rendering_Provider', 'enc_nbr']].drop_duplicates()

In [ ]:
dx_codes_for_enc = df[['enc_nbr', 'dxCode']].drop_duplicates().sort_values(['enc_nbr', 'dxCode'])

In [ ]:
#dx_codes_for_enc.head()

In [ ]:
dx_codes_for_enc.rename(columns={'dxCode':'dx1'}, inplace=True)
dx_codes_for_enc['dx1'] = dx_codes_for_enc['dx1'].str.replace('.', '')

In [ ]:
sas.dataframe2sasdata(df, 'df')
sas.dataframe2sasdata(dx_codes_for_enc, 'dx_codes_for_enc')

In [ ]:
%%SAS sas

libname pe 'C:\Users\mregine\Desktop\ccs_build' ; 

data pe.Endocrinology;
set df;
run;

data pe.dx_codes_for_enc;
set dx_codes_for_enc;
run;

In [ ]:
%%SAS sas

LIBNAME  IN1     'C:\Users\mregine\Desktop\ccs_build\in';                 * Location of input discharge data;
LIBNAME  OUT1    'C:\Users\mregine\Desktop\ccs_build\out';                * Location of output data;

data in1.df;
set pe.Dx_codes_for_enc;
run;



/******************************************************************/
/* Title:       ICD-10-CM/PCS CCS MULTI-LEVEL LOAD SOFTWARE		  */
/*                                                                */
/* PROGRAM:     ICD10CMPCS_MULTI_CCS_LOAD_PROGRAM.SAS             */
/*                                                                */
/* Description: This program creates multi-level ICD-10-CM/PCS    */
/*              CCS categories for data using ICD-10-CM/PCS       */
/*              diagnosis or procedure codes. The multi-level CCS */
/*              categories are an expansion of the single-level   */
/*              ICD-10-CM/PCS CCS categories.                     */
/*																  */
/*              There are two general sections to this program:   */
/*                                                                */
/*              1) The first section creates temporary SAS        */
/*                 informats using ICD-10-CM and PCS tool files.  */
/*                 The multi-level categories are located in      */
/*                 columns 5-8.                                   */
/*                 These informats are used in step 2 to create   */
/*                 the multi-level CCS variables. To save         */
/*                 space, we used macros to call the same code    */
/*                 repeatedly for the two formats.                */
/*              2) The second section loops through the diagnosis */
/*                 and/or procedure arrays on your SAS dataset    */
/*                 and assigns the multi-level CCS categories.    */
/*                                                                */
/******************************************************************/
* Path & name of the ICD-10-CM and PCS tool files ;
FILENAME INRAW1  'C:\Users\mregine\Desktop\ccs_build\MultiCCSLoadPrograms\ccs_dx_icd10cm_2019_1(1)\ccs_dx_icd10cm_2019_1.CSV' LRECL=300;  
FILENAME INRAW2  'C:\Users\mregine\Desktop\ccs_build\MultiCCSLoadPrograms\ccs_pr_icd10pcs_2019_1\ccs_pr_icd10pcs_2019_1.CSV' LRECL=300;   


TITLE1 'CREATE ICD-10-CM/PCS CCS MULTI-LEVEL TOOL CATEGORIES';
TITLE2 'USE WITH DISCHARGE ADMINISTRATIVE DATA THAT HAS ICD-10-CM OR PCS CODES';

/******************************************************************/
/*  Macro Variables that must be set to define the characteristics*/
/*  of your SAS discharge data. Change these values to match the  */
/*  number of diagnoses and procedures in your dataset. Change    */
/*  CORE to match the name of your dataset.                       */
/******************************************************************/
* Maximum number of DXs on any record;        %LET NUMDX=1;
* Maximum number of PRs on any record;        %LET NUMPR=0;
* Input SAS file member name;                 %LET CORE=df;

%Macro MultiCCS;
/******************* SECTION 1: CREATE INFORMATS ******************/
/*  SAS Load the ICD-10-CM/PCS CCS multi-level CM and PCS tools   */
/*  and convert them into temporary SAS informats used to assign  */
/*  the multi-level CCS variables in the next step.               */
/******************************************************************/

%macro multidxccs(fmt_,var1_,var2_,var3_,var4_,var5_,var6_,var7_);
DATA CCS_MULTI_DX;
   INFILE INRAW1 DSD DLM=',' END = EOF FIRSTOBS=2;
	INPUT
	   START       : $CHAR7.
	   &var1_      : $CHAR4.
	   &var2_      : $CHAR100.
	   &var3_      : $CHAR100.
	   &var4_      : $CHAR2.
	   &var5_      : $CHAR100.
	   &var6_      : $CHAR5.
	   &var7_      : $CHAR100.
	;
   RETAIN HLO " ";
   FMTNAME = "&fmt_" ;
   TYPE    = "J" ;
   OUTPUT;

   IF EOF THEN DO ;
      START = " " ;
	  LABEL = " " ;
      HLO   = "O";
      OUTPUT ;
   END ;
RUN;

PROC FORMAT LIB=WORK CNTLIN = CCS_MULTI_DX;
RUN;
%mend multidxccs;

%if &NUMDX > 0 %then %do;
%multidxccs($L1DCCS,SL,CL,SLL,LABEL,L1L,L2,L2L);
%multidxccs($L2DCCS,SL,CL,SLL,L1,L1L,LABEL,L2L);
%end;

%macro multiprccs(fmt_,var1_,var2_,var3_,var4_,var5_,var6_,var7_);
DATA CCS_MULTI_PR ;
   INFILE INRAW2 DSD DLM=',' END = EOF FIRSTOBS=2;
	INPUT
	   START       : $CHAR7.
	   &var1_      : $CHAR3.
	   &var2_      : $CHAR100.
	   &var3_      : $CHAR100.
	   &var4_      : $CHAR2.
	   &var5_      : $CHAR100.
	   &var6_      : $CHAR5.
	   &var7_      : $CHAR100.
	;
   RETAIN HLO " ";
   FMTNAME = "&fmt_" ;
   TYPE    = "J" ;
   OUTPUT;

   IF EOF THEN DO ;
      START = " " ;
		LABEL = " " ;
      HLO   = "O";
      OUTPUT ;
   END ;
RUN;

PROC FORMAT LIB=WORK CNTLIN = CCS_MULTI_PR ;
RUN;
%mend multiprccs;

%if &NUMPR > 0 %then %do;
%multiprccs($L1PCCS,SL,CL,SLL,LABEL,L1L,L2,L2L);
%multiprccs($L2PCCS,SL,CL,SLL,L1,L1L,LABEL,L2L);
%end;


/*********** SECTION 2: CREATE ICD-10-CM/PCS MULTI-LEVEL CCS CATS ********/
/*  Create multi-level CCS categories for CM/PCS using the SAS           */
/*  informats created above & the SAS file you wish to augment.          */
/*  Users can change the names of the output CCS variables if            */
/*  needed here. It is also important to make sure that the              */
/*  correct ICD-10-CM/PCS diagnosis or procedure names from your SAS     */
/*  file are used in the arrays 'DXS' and 'PRS'.                         */
/*************************************************************************/  

DATA OUT1.NEW_MULTI_CCS (DROP = i);
  SET IN1.&CORE;

  %if &NUMDX > 0 %then %do;
  ARRAY L1DCCS  (*)   $5 L1DCCS1-L1DCCS&NUMDX;   * Suggested name for ICD-10-CM Level 1 Multi-Level DX CCS variables;
  ARRAY L2DCCS  (*)   $5 L2DCCS1-L2DCCS&NUMDX;   * Suggested name for ICD-10-CM Level 2 Multi-Level DX CCS variables;
  ARRAY DXS     (*)   $  DX1-DX&NUMDX;           * Change ICD-10-CM diagnosis variable names to match your file;
  %end;

  %if &NUMPR > 0 %then %do;
  ARRAY L1PCCS  (*)   $5 L1PCCS1-L1PCCS&NUMPR;   * Suggested name for ICD-10-PCS Level 1 Multi-Level PR CCS variables;
  ARRAY L2PCCS  (*)   $5 L2PCCS1-L2PCCS&NUMPR;   * Suggested name for ICD-10-PCS Level 2 Multi-Level PR CCS variables;
  ARRAY PRS     (*)   $  PR1-PR&NUMPR;           * Change ICD-10-PCS procedure variable names to match your file;
  %end;
 
  /***************************************************/
  /*  Loop through the CM diagnosis array in your SAS*/
  /*  dataset and create the multi-level diagnosis   */
  /*  CCS variables.                                 */
  /***************************************************/
  %if &NUMDX > 0 %then %do;
  DO I = 1 TO &NUMDX;
	 L1DCCS(I) = INPUT(DXS(I),$L1DCCS.);
	 L2DCCS(I) = INPUT(DXS(I),$L2DCCS.);
  END;  
  %end;

  /***************************************************/
  /*  Loop through the PCS procedure array in your   */
  /*  SAS dataset & create the multi-level procedure */
  /*  CCS variables.                                 */
  /***************************************************/
  %if &NUMPR > 0 %then %do;
  DO I = 1 TO &NUMPR;
	 L1PCCS(I) = INPUT(PRS(I),$L1PCCS.);
	 L2PCCS(I) = INPUT(PRS(I),$L2PCCS.);
  END;  
  %end;

RUN;

PROC PRINT DATA=OUT1.NEW_MULTI_CCS (OBS=10);
  %if &NUMDX > 0 %then %do;
     VAR  DX1 L1DCCS1 L2DCCS1;
  %end;
  %else %if &NUMPR > 0 %then %do;
     VAR PR1 L1PCCS1 L2PCCS1;
  %end;
  title2 "Partial Print of the Output ICD-10-CM/PCS Multi-Level CCS File";
RUN;
%Mend MultiCCS;
%MultiCCS;

In [ ]:
%%SAS sas

/******************************************************************/
/* Title:       ICD-10-CM/PCS CCS SUMMARY STATISTICS PROGRAM      */
/*              SOFTWARE			                              */
/*                                                                */
/* PROGRAM:     ICD10CMPCS_MULTI_CCS_SUMMARY_PROGRAM.SAS          */
/*                                                                */
/* Description: This program prints a report that shows the number*/
/*              and percent of discharges for each ICD-10-CM/PCS  */
/*              CCS Category. The report works off of the primary */
/*              ICD-10-CM diagnosis or the primary ICD-10-PCS     */
/*              procedure and should be used after the Multi-     */
/*              Level CCS DX Categories are assigned by the       */
/*              ICD-10-CM/PCS CCS Load Program called             */
/*              (ICD10CMPCS_Multi_CCS_Load_Program.sas).          */
/******************************************************************/

OPTIONS nodate FormChar='|----|+|---+=|#/\<>*' linesize=max;

FILENAME INRAW1  'C:\Users\mregine\Desktop\ccs_build\MultiCCSLoadPrograms\DXMLABEL10.CSV';        * Path & name of DX CCS Category Labels file ;
FILENAME INRAW2  'C:\Users\mregine\Desktop\ccs_build\MultiCCSLoadPrograms\PRMLABEL10.CSV';        * Path & name of PR CCS Category Labels file ;

/*******************************************************************/
/*  Macro Variables that must be set to define the characteristics */
/*  of your SAS discharge data. Change these values to indicate    */
/*  the name of your SAS file and whether you want to report on    */
/*  diagnoses and/or procedures.                                   */
/*******************************************************************/
* Input SAS file member name;                                    %LET CORE=NEW_MULTI_CCS;
* Run Report on Primary Diagnosis or Procedure? ('DX'/'PR');     %LET RPT = 'DX';


%Macro CCSFreq;
/******************* CREATE LABEL FORMAT ********************************/
/*  SAS Load the ICD-10-CM/PCS CCS multi-level categories and labels    */
/*  and create a SAS format that will be used to print the Multi-Level  */
/*  CCS category label in the report.                                   */
/************************************************************************/

DATA LABELS;
   %if &RPT='DX' %then %do;   
   INFILE INRAW1 DSD DLM=',' END = EOF FIRSTOBS=2;
	%end;
   %else %if &RPT='PR' %then %do; 
	INFILE INRAW2 DSD DLM=',' END = EOF FIRSTOBS=2;
   %end;
	INPUT
	   START      : $9.
	   LABEL      : $99.
	;
   RETAIN HLO " ";
   FMTNAME = "$CCSLBL" ;
   TYPE    = "C" ;
   OUTPUT;

   IF EOF THEN DO ;
      START = " " ;
		LABEL = " " ;
      HLO   = "O";
      OUTPUT ;
   END ;
RUN;

PROC FORMAT LIB=WORK  CNTLIN = LABELS ;
RUN;


/************************ GET STATISTICS ****************************/
/*  Run frequencies on your SAS file containing the multi-level DX  */
/*  and/or PR CCS variables. Capture the frequencies into files and */
/*  combine them for reporting. The frequencies use the variable    */
/*  names for the multi-level primary DX/PR that are created in the */
/*  CCS Load program. You may need to change the variable names if  */
/*  you use custom code to create your multi-level CCS variables.   */
/********************************************************************/
PROC FREQ DATA=OUT1.&CORE;
    %if &RPT='DX' %then %do;
	   TABLES L1DCCS1 / MISSING LIST NOPRINT OUT=DXSUM1;
       TABLES L2DCCS1 / MISSING LIST NOPRINT OUT=DXSUM2;
	 %end;
    %else %if &RPT='PR' %then %do;
       TABLES L1PCCS1 / MISSING LIST NOPRINT OUT=PRSUM1;
       TABLES L2PCCS1 / MISSING LIST NOPRINT OUT=PRSUM2;
	 %end;
RUN;

DATA SUMMARY1 (KEEP=VAR1 COUNT PERCENT);
   SET 
	   %if &RPT='DX' %then %do;
	   DXSUM1 DXSUM2
		%end;
      %else %if &RPT='PR' %then %do;	
		PRSUM1 PRSUM2
		%end;
		;
	LENGTH VAR1 $9.; 
	%if &RPT='DX' %then %do;
	IF L1DCCS1 NE '' THEN DO;
	   VAR1=L1DCCS1;
		OUTPUT;
   END;
	ELSE IF L2DCCS1 NE '' THEN DO;
	   VAR1=L2DCCS1;
		OUTPUT;
	END;
   %end;
	%else %if &RPT='PR' %then %do;
	IF L1PCCS1 NE '' THEN DO;
	   VAR1=L1PCCS1;
	   OUTPUT;
   END;
	ELSE IF L2PCCS1 NE '' THEN DO;
	   VAR1=L2PCCS1;
		OUTPUT;
	END;
   %end;
RUN;


/************************ MAKE SORT ORDER/LABELS ********************/
/*  Take the file of multi-level CCS frequencies and sort it in     */
/*  proper order. Create CCS Category labels using the format       */
/*  made earlier. Run Proc Report to get the output.                */
/********************************************************************/
DATA SUMMARY2 ;
   LENGTH SORT1-SORT2 $3. 
   CCSLABEL $90. ;
   SET SUMMARY1;
   SORT1=INPUT(SCAN(VAR1,1),2.);
   SORT2=INPUT(SCAN(VAR1,2),2.);
	 CCSLABEL = PUT(VAR1,$CCSLBL.);
RUN;

PROC SORT DATA=SUMMARY2 ;
   BY SORT1 SORT2;
RUN;

PROC REPORT DATA=SUMMARY2 HEADLINE  ;
   COLUMN VAR1 CCSLABEL COUNT PERCENT;
	DEFINE VAR1 / DISPLAY 'CCS Category' ;
	DEFINE CCSLABEL / DISPLAY 'CCS Label';
	DEFINE COUNT / ANALYSIS width=10  'Number of Discharges';
	DEFINE PERCENT / ANALYSIS width=10 FORMAT=4.1 '% of Discharges';
	TITLE1 "CCS SOFTWARE";
	%if &RPT='DX' %then %do;
	   TITLE2 "Diagnosis (CCS category number and name)";
	%end;
	%else %if &RPT='PR' %then %do;
	   TITLE2 "Procedure (CCS category number and name)";
	%end;
RUN;
%Mend CCSFreq;
%CCSFreq;


In [ ]:
New_multi_ccs = sas.sasdata2dataframe('New_multi_ccs', libref='out1')
Ccs_multi_dx = sas.sasdata2dataframe('Ccs_multi_dx')

In [ ]:
sas.endsas()